# A/B Test Challenge



---

#### What is an A/B Test? 

It is a decision making support & research methodology that allow you to measure an impact of a change in a product (e.g.: a digital product). For this challenge you will analyse the data resulting of an A/B test performed on a digital product where a new set of sponsored ads are included.


#### Measure of success

Metrics are needed to measure the success of your product. They are typically split in the following categories: 

- __Enganged based metrics:__ number of users, number of downloads, number of active users, user retention, etc.

- __Revenue and monetization metrics:__ ads and affiliate links, subscription-based, in-app purchases, etc.

- __Technical metrics:__ service level indicators (uptime of the app, downtime of the app, latency).



---

## Metrics understanding

In this part you must analyse the metrics involved in the test. We will focus in the following metrics:

- Activity level + Daily active users (DAU).

- Click-through rate (CTR)

### Activity level

In the following part you must perform every calculation you consider necessary in order to answer the following questions:

- How many activity levels you can find in the dataset (Activity level of zero means no activity).

- What is the amount of users for each activity level.

- How many activity levels do you have per day and how many records per each activity level.

At the end of this section you must provide your conclusions about the _activity level_ of the users.

__Dataset:__ `activity_pretest.csv`

In [3]:
# your-code
#Cual es la cantidad de actividad por usuario que existe por esa actividad? valuecount, groupby
#Cuantos niveles de actividad tienes por día y cuántos registros
#El data set original cuantas filas tendría? Cual sería la operación para calcular las filas del dataset original - 



In [3]:
import pandas as pd
import os
from statsmodels.stats.weightstats import ztest
from scipy import stats
import seaborn as sns
import matplotlib.pylab as plt

In [3]:
# Import datasets
content = os.listdir('./data/')

# With this function I can extract all of datasets in diferent dataframes. In order to understand each dataframe, I will 
# print the name, columns name and shape of each dataframe.
for name_file in content:
    name_df = name_file.split('.')[0] + '_df'
    exec(f'{name_df} = pd.read_csv(f"./data/{name_file}", sep=",")')
    exec(f'columns_name = {name_df}.columns.values')
    exec(f'shape = {name_df}.shape')
    print(name_df,'| columns names: ', columns_name, '| df shape: ', shape)

ctr_pretest_df | columns names:  ['userid' 'dt' 'ctr'] | df shape:  (950875, 3)
activity_all_df | columns names:  ['userid' 'dt' 'groupid' 'activity_level'] | df shape:  (3660000, 4)
ctr_all_df | columns names:  ['userid' 'dt' 'groupid' 'ctr'] | df shape:  (2303408, 4)
activity_pretest_df | columns names:  ['userid' 'dt' 'activity_level'] | df shape:  (1860000, 3)


In [4]:
# How many activity levels you can find in the dataset (Activity level of zero means no activity).

len(activity_pretest_df['activity_level'].unique())

21

In [5]:
activity_pretest_df

,userid,dt,activity_level
0,a5b70ae7-f07c-4773-9df4-ce112bc9dc48,2021-10-01,0
1,d2646662-269f-49de-aab1-8776afced9a3,2021-10-01,0
2,c4d1cfa8-283d-49ad-a894-90aedc39c798,2021-10-01,0
3,6889f87f-5356-4904-a35a-6ea5020011db,2021-10-01,0
4,dbee604c-474a-4c9d-b013-508e5a0e3059,2021-10-01,0
...,...,...,...
1859995,200d65e6-b1ce-4a47-8c2b-946db5c5a3a0,2021-10-31,20
1859996,535dafe4-de7c-4b56-acf6-aa94f21653bc,2021-10-31,20
1859997,0428ca3c-e666-4ef4-8588-3a2af904a123,2021-10-31,20
1859998,a8cd1579-44d4-48b3-b3d6-47ae5197dbc6,2021-10-31,20


In [11]:
# user activity

user_activity = pd.DataFrame(activity_pretest_df['activity_level'].value_counts()).reset_index()
user_activity.columns = ['activity_level','num_users']
user_activity['proportions'] = user_activity['num_users'] / user_activity['num_users'].sum()
user_activity['proportions'] = user_activity['proportions'].round(2)
user_activity

#El 50% de los usuarios no tiene actividad, y el otro porcentaje está distribuido parecido en el resto de niveles excepto los de 20

,activity_level,num_users,proportions
0,0,909125,0.49
1,5,49227,0.03
2,2,49074,0.03
3,18,48982,0.03
4,10,48943,0.03
5,16,48934,0.03
6,12,48911,0.03
7,6,48901,0.03
8,19,48901,0.03
9,11,48832,0.03


In [10]:
len(activity_pretest_df['userid'].unique())

60000

In [18]:
# unique activity levels

activity_pretest_df['activity_level'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20])

In [14]:
# time delta

activity_pretest_df['dt'].max() - activity_pretest_df['dt'].min()

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [17]:
# activity levels per day

act_pretest_day = activity_pretest_df.groupby(['dt'])[['activity_level']].nunique().reset_index()
act_pretest_day

,dt,activity_level
0,2021-10-01,21
1,2021-10-02,21
2,2021-10-03,21
3,2021-10-04,21
4,2021-10-05,21
5,2021-10-06,21
6,2021-10-07,21
7,2021-10-08,21
8,2021-10-09,21
9,2021-10-10,21


In [ ]:
# Todos los días hay todos los niveles de usuario

### Daily active users (DAU)

![ab_test](./img/user_activity_ab_testing.JPG)


The daily active users (DAU) refers to the amount of users that are active per day (activity level of zero means no activity). You must perform the calculation of this metric and provide your insights about it.

__Dataset:__ `activity_pretest.csv`

In [5]:
# your-code
# De esos 60.000 usuarios cuántos están activos cada día. 

#Te cargas los del 0 y con los que te quedas haces un nunique y con esto sabes que más o menos tu web tiene en torno a los 60 000 usuarios



In [21]:
# DAU

act_pretest_ok = activity_pretest_df[activity_pretest_df['activity_level'] != '0']
dau_pretest = act_pretest_ok.groupby(['dt'])[['userid']].nunique().reset_index()
dau_pretest.columns = ['timestamp','dau']
dau_pretest

,timestamp,dau
0,2021-10-01,60000
1,2021-10-02,60000
2,2021-10-03,60000
3,2021-10-04,60000
4,2021-10-05,60000
5,2021-10-06,60000
6,2021-10-07,60000
7,2021-10-08,60000
8,2021-10-09,60000
9,2021-10-10,60000


In [24]:
act_pretest_ok = activity_pretest_df[activity_pretest_df['activity_level']]

KeyError: "None of [Index([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,\n       ...\n       20, 20, 20, 20, 20, 20, 20, 20, 20, 20],\n      dtype='int64', length=1860000)] are in the [columns]"

### Click-through rate (CTR)

![ab_test](./img/ad_click_through_rate_ab_testing.JPG)

Click-through rate (CTR) refers to the percentage of clicks that the user perform from the total amount ads showed to that user during a certain day. You must perform the analysis of this metric (e.g.: average CTR per day) and provide your insights about it.

__Dataset:__ `ctr_pretest.csv`

In [29]:
# your-code


# CTR
ctr_pretest = pd.read_csv('./data/ctr_pretest.csv')
ctr_pretest

# 34.28 es un porcentaje, ese usuario le da click al 34% de los anuncios

,userid,dt,ctr
0,4b328144-df4b-47b1-a804-09834942dce0,2021-10-01,34.28
1,34ace777-5e9d-40b3-a859-4145d0c35c8d,2021-10-01,34.67
2,8028cccf-19c3-4c0e-b5b2-e707e15d2d83,2021-10-01,34.77
3,652b3c9c-5e29-4bf0-9373-924687b1567e,2021-10-01,35.42
4,45b57434-4666-4b57-9798-35489dc1092a,2021-10-01,35.04
...,...,...,...
950870,a09a3687-b71a-4a67-b1ef-9b05c9770c4c,2021-10-31,32.33
950871,c843a595-b94c-42e1-b2fe-ec096070681e,2021-10-31,30.09
950872,edcdf0c1-3d8f-47e8-b7dd-05505749eb69,2021-10-31,35.71
950873,76b7a9ae-98fa-4c77-869d-594a4ef7282d,2021-10-31,34.76


In [30]:
# CTR per day
ctr_pretest_day = ctr_pretest.groupby(['dt'])[['ctr']].mean().reset_index()
ctr_pretest_day

,dt,ctr
0,2021-10-01,32.993446
1,2021-10-02,32.991664
2,2021-10-03,32.995086
3,2021-10-04,32.992995
4,2021-10-05,33.004375
5,2021-10-06,33.018564
6,2021-10-07,32.988500
7,2021-10-08,32.998654
8,2021-10-09,33.005082
9,2021-10-10,33.007134


In [31]:
# CTR mean
ctr_pretest_mean = ctr_pretest_day['ctr'].mean()
ctr_pretest_mean

33.00024304382363

---

## Pretest metrics 

In this section you will perform the analysis of the metrics using the dataset that includes the result for the test and control groups, but only for the pretest data (i.e.: prior to November 1st, 2021). You must provide insights about the metrics (__Activity level__, __DAU__ and __CTR__) and also perform an hyphotesis test in order to determine whether there is any statistical significant difference between the groups prior to the start of the experiment. You must try different approaches (i.e.: __z-test__ and __t-test__) and compare the results.


__Datasets:__ `activity_all.csv`, `ctr_all.csv`

In [12]:
# your-code
activity_all = pd.read_csv('./data/activity_all.csv')
activity_all
ctr_all = pd.read_csv('./data/ctr_all.csv')
ctr_all

,userid,dt,groupid,ctr
0,60389fa7-2d71-4cdf-831c-c2bb277ffa1e,2021-11-13,0,31.81
1,b59cb225-d160-4851-92d2-7cc8120a2f63,2021-11-13,0,30.46
2,aa336050-934e-453f-a5b0-dd881fcd114e,2021-11-13,0,34.25
3,8df767f4-a10f-4322-a722-676b7e02b372,2021-11-13,0,34.92
4,a74762ed-4da0-42ab-91d2-40d7e808dfe9,2021-11-13,0,34.95
...,...,...,...,...
2303403,932e0348-ea2d-4b98-8782-aa84420f0796,2021-11-12,1,37.27
2303404,6775a825-6d3d-4dc3-9335-cad061736752,2021-11-12,1,39.14
2303405,a7b55365-21f1-4123-b2b5-485a8c7b98da,2021-11-12,1,40.05
2303406,a6fa937c-6f40-4f04-b15b-f1de09e179db,2021-11-12,1,38.14


In [13]:
acti_pre = activity_all.loc[activity_all['dt'] <= '2021-10-30',:]
acti_pre1 = acti_pre.loc[acti_pre['groupid'] == 1,:]
acti_pre2 = acti_pre.loc[acti_pre['groupid'] == 0,:]
print(acti_pre1.info())
print(acti_pre2.info())

<class 'pandas.core.frame.DataFrame'>
Index: 901470 entries, 2 to 3624660
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   userid          901470 non-null  object
 1   dt              901470 non-null  object
 2   groupid         901470 non-null  int64 
 3   activity_level  901470 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 34.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
Index: 898530 entries, 0 to 3624659
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   userid          898530 non-null  object
 1   dt              898530 non-null  object
 2   groupid         898530 non-null  int64 
 3   activity_level  898530 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 34.3+ MB
None


In [14]:
dau_pre1 = acti_pre1.loc[acti_pre1['activity_level'] != 0,:].groupby('dt').count()['userid']
dau_pre2 = acti_pre2.loc[acti_pre2['activity_level'] != 0,:].groupby('dt').count()['userid']
dau_pre1_mean = dau_pre1.mean()
dau_pre1_mean
dau_pre2_mean = dau_pre2.mean()
dau_pre2_mean

15324.633333333333

In [15]:
z_stat, p_value = ztest(dau_pre1, value=dau_pre1_mean)

# Imprimir resultados
print(f"Estadístico Z: {z_stat}")
print(f"Valor p: {p_value}")

# Tomar una decisión basada en el valor p
alpha = 0.05
if p_value < alpha:
    print("Rechazamos la hipótesis nula")
else:
    print("No podemos rechazar la hipótesis nula")

Estadístico Z: 0.0
Valor p: 1.0
No podemos rechazar la hipótesis nula


In [16]:
z_stat, p_value = ztest(dau_pre2, value=dau_pre2_mean)

# Imprimir resultados
print(f"Estadístico Z: {z_stat}")
print(f"Valor p: {p_value}")

# Tomar una decisión basada en el valor p
alpha = 0.05
if p_value < alpha:
    print("Rechazamos la hipótesis nula")
else:
    print("No podemos rechazar la hipótesis nula")

Estadístico Z: 0.0
Valor p: 1.0
No podemos rechazar la hipótesis nula


---

## Experiment metrics 

In this section you must perform the same analysis as in the previous section, but using the data generated during the experiment (i.e.: after November 1st, 2021). You must provide insights about the metrics (__Activity level__, __DAU__ and __CTR__) and also perform an hyphotesis test in order to determine whether there is any statistical significant difference between the groups during the experiment. You must try different approaches (i.e.: __z-test__ and __t-test__) and compare the results.


__Datasets:__ `activity_all.csv`, `ctr_all.csv`

In [10]:
# your-code

activity_all = pd.read_csv('./data/activity_all.csv')
activity_all


,userid,dt,groupid,activity_level
0,a5b70ae7-f07c-4773-9df4-ce112bc9dc48,2021-10-01,0,0
1,d2646662-269f-49de-aab1-8776afced9a3,2021-10-01,0,0
2,c4d1cfa8-283d-49ad-a894-90aedc39c798,2021-10-01,1,0
3,6889f87f-5356-4904-a35a-6ea5020011db,2021-10-01,0,0
4,dbee604c-474a-4c9d-b013-508e5a0e3059,2021-10-01,1,0
...,...,...,...,...
3659995,f0126b50-ad74-4480-9250-41b50a408932,2021-11-30,0,20
3659996,6ffe1efe-2e5d-427f-95ff-cc862c46c798,2021-11-30,1,20
3659997,f2073207-25dd-4127-a893-b70106d5ead7,2021-11-30,0,20
3659998,0416f2be-3ab8-481b-873c-3678b4705ecf,2021-11-30,1,20


In [17]:
ctr_all = pd.read_csv('./data/ctr_all.csv')
ctr_all

,userid,dt,groupid,ctr
0,60389fa7-2d71-4cdf-831c-c2bb277ffa1e,2021-11-13,0,31.81
1,b59cb225-d160-4851-92d2-7cc8120a2f63,2021-11-13,0,30.46
2,aa336050-934e-453f-a5b0-dd881fcd114e,2021-11-13,0,34.25
3,8df767f4-a10f-4322-a722-676b7e02b372,2021-11-13,0,34.92
4,a74762ed-4da0-42ab-91d2-40d7e808dfe9,2021-11-13,0,34.95
...,...,...,...,...
2303403,932e0348-ea2d-4b98-8782-aa84420f0796,2021-11-12,1,37.27
2303404,6775a825-6d3d-4dc3-9335-cad061736752,2021-11-12,1,39.14
2303405,a7b55365-21f1-4123-b2b5-485a8c7b98da,2021-11-12,1,40.05
2303406,a6fa937c-6f40-4f04-b15b-f1de09e179db,2021-11-12,1,38.14


In [18]:
# your-code
df_experimento = activity_all[activity_all['dt'] > '2021-11-01']
df_experimento
ctr_all

,userid,dt,groupid,ctr
0,60389fa7-2d71-4cdf-831c-c2bb277ffa1e,2021-11-13,0,31.81
1,b59cb225-d160-4851-92d2-7cc8120a2f63,2021-11-13,0,30.46
2,aa336050-934e-453f-a5b0-dd881fcd114e,2021-11-13,0,34.25
3,8df767f4-a10f-4322-a722-676b7e02b372,2021-11-13,0,34.92
4,a74762ed-4da0-42ab-91d2-40d7e808dfe9,2021-11-13,0,34.95
...,...,...,...,...
2303403,932e0348-ea2d-4b98-8782-aa84420f0796,2021-11-12,1,37.27
2303404,6775a825-6d3d-4dc3-9335-cad061736752,2021-11-12,1,39.14
2303405,a7b55365-21f1-4123-b2b5-485a8c7b98da,2021-11-12,1,40.05
2303406,a6fa937c-6f40-4f04-b15b-f1de09e179db,2021-11-12,1,38.14


In [19]:
exp_group = df_experimento[df_experimento['groupid'] == 1]  
exp_control = df_experimento[df_experimento['groupid'] == 0] 

In [20]:
exp_activity_test = exp_group['activity_level']
exp_activity_control = exp_control['activity_level']

In [21]:
#dau
exp_dau_test = exp_group.groupby('dt')['userid'].nunique()
exp_dau_control = ctr_all.groupby('dt')['userid'].nunique()
mean = exp_dau_test.mean()
mean

30049.0

In [ ]:
z_stat, p_value = ztest(df_experimento, value=mean)

# Imprimir resultados
print(f"Estadístico Z: {z_stat}")
print(f"Valor p: {p_value}")

# Tomar una decisión basada en el valor p
alpha = 0.05
if p_value < alpha:
    print("Rechazamos la hipótesis nula")
else:
    print("No podemos rechazar la hipótesis nula")

---

## Conclusions

Please provide your conclusions after the analyses and your recommendation whether we may or may not implement the changes in the digital product.

In [8]:
# your-conclusions

#1º caso: no podemos rechazar la H1
#2º caso: no podemos rechazar la H1
#3º caso: podemos rechazar la H1
#4º caso: podemos rechazar la H1


---